# FIMOC: Fiber mode online calculator

In [12]:
## Imports

import time

import numpy as np
import scipy.special as spe
from scipy import optimize

import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, Label, IntSlider, FloatSlider, Button, HTML, Output, FloatText, Box

%matplotlib inline
plt.style.use("bmh")

pi = np.pi

## Initialization
dpi = 100
plt.rcParams["figure.dpi"] = dpi

mesh_limit = 2.
xx = np.linspace(-mesh_limit, mesh_limit, 60)
yy = np.linspace(-mesh_limit, mesh_limit, 60)

x_mesh, y_mesh = np.meshgrid(xx, yy)
r_mesh = np.sqrt(x_mesh ** 2 + y_mesh ** 2)
phi_mesh = np.arctan2(y_mesh, x_mesh)

ones_mesh = np.ones((len(xx), len(yy)))
zeros_mesh = np.zeros((len(xx), len(yy)))

in_core_mesh = ones_mesh.copy()
in_core_mesh[r_mesh > 1] = zeros_mesh[r_mesh > 1]  # mask core with ones

in_clad_mesh = ones_mesh.copy()
in_clad_mesh[r_mesh <= 1] = zeros_mesh[r_mesh <= 1]  # mask cladding with ones

# this is to plot the core perimeter later
phi_core_shape = np.linspace(0, 2 * pi, 60)
x_core_shape = 1 * np.cos(phi_core_shape)
y_core_shape = 1 * np.sin(phi_core_shape)

n_cladding = 1.444

## Functions

def zero_func(X, V, L):
    Y = np.sqrt(V ** 2 - X ** 2)
    return X * spe.jv(L + 1, X) / spe.jv(L, X) - Y * spe.kv(L + 1, Y) / spe.kv(L, Y)


def find_zeros_exact(X, Y, V, L):
    f = X * spe.jv(L + 1, X) / spe.jv(L, X) - Y * spe.kv(L + 1, Y) / spe.kv(L, Y)

    tt = len(X)
    zeros = []
    brackets = []

    for ii in range(tt - 1):
        
        if f[ii] * f[ii + 1] < 0:  # change of sign
            
            if ii != 0 and ii != tt - 2:  # not at an extreme
                
                if abs(f[ii - 1] - f[ii + 2]) > abs(f[ii] - f[ii + 1]):  # not an asymptote
                    brackets += [[X[ii], X[ii + 1]]]
                    
            else:
                brackets += [[X[ii], X[ii + 1]]]
    
    sols = []
    
    for br in brackets:
        optimum = optimize.root_scalar(zero_func, args=(V, L), bracket=br, method='brentq')
        sols.append(optimum)

    return [a.root for a in sols]


def calc_Er(mode,r):
    kt = mode.X / mode.a
    gamma = np.sqrt(mode.V ** 2 - mode.X ** 2) / mode.a
    Er = np.empty([len(r)])
    
    correction_clad = spe.jv(mode.L, kt * mode.a) / spe.kv(mode.L, gamma * mode.a)
    
    for ii in range(len(r)):
        
        if r[ii] < - mode.a:
            Er[ii] = spe.kv(mode.L, gamma * abs(r[ii])) * correction_clad
            if mode.L % 2:
                Er[ii] = -Er[ii]
                
        elif r[ii] > mode.a:
            Er[ii] = spe.kv(mode.L, gamma * r[ii]) * correction_clad
            
        else:
            Er[ii] = spe.jv(mode.L, kt * r[ii])
    
    Er = Er / max(abs(Er))
    return Er

def calc_E_mesh(mode,r_mesh,phi_mesh,in_core_mesh,in_clad_mesh):
    X = mode.X
    L = mode.L
    V = mode.V
    a = mode.a
    kt = X / a
    gamma = np.sqrt(V ** 2 - X ** 2) / a

    E_core = spe.jv(L, kt * a * r_mesh) * np.cos(L * phi_mesh)
    E_clad = spe.kv(L, gamma * a * r_mesh) * np.cos(L * phi_mesh) / spe.kv(L, gamma * a) * spe.jv(L, kt * a)
    E = E_core * in_core_mesh + E_clad * in_clad_mesh
    E = E / np.amax(E)
    return E

class Mode:
    pass


def find_modes(a=8.2 / 2, Na=0.12, n_cladding=1.444, w=1.55):
    """Calculates all the modes in the fiber, and puts them in the list of modes.
    It also returns the total number of modes, which is higher than len(modes) because some modes
    have degeneracy 2 (L=0) and some have degeneracy 4 (L>0)"""

    k0 = 2 * pi / w
    n_core = np.sqrt(Na ** 2 + n_cladding ** 2)

    V = k0 * a * Na
    phi = np.linspace(1E-10, pi / 2 - 1E-10, 5000)
    X = V * np.sin(phi)
    Y = V * np.cos(phi)

    L = 0
    M = 1
    modes = []
    tot_modes = 0
    
    with np.errstate(invalid='ignore'):
        while True:
            sols = find_zeros_exact(X, Y, V, L)
        
            for sol in sols:
                mode = Mode()
                mode.X, mode.L, mode.M, mode.V, mode.a = sol, L, M, V, a

                mode.neff = np.sqrt(n_core**2 - (sol/(a*k0)) ** 2)
                mode.label = f"LP({L},{M})"

                mode.degeneracy = 2 if L == 0 else 4

                modes.append(mode)

                tot_modes += mode.degeneracy

                M += 1

            M = 1
            L += 1

            if len(sols) == 0:
                break
            
    return modes, tot_modes


## Interactive elements

## Sliders
slider_diam = FloatSlider(min=2.0, max=80, value=50)
slider_Na = FloatSlider(min=0.02, max=0.5, step=0.001, value=0.12)
slider_lambda = FloatSlider(min=0.5, max=2.0, step=0.01, value=1.55)
text_n_clad = FloatText(min = 1.0, max = 2.0, step = 0.01, value = 1.444, layout=Layout(width="80px"))
text_ng_clad = FloatText(min = 1.0, max = 2.0, step = 0.01, value = 1.4626, layout=Layout(width="80px"))


## Labels
label_n_core = Label()

label_V = Label()

label_V_max = Label(value = r'\(\color{red} {V max = 30!}\)')
label_V_max.layout.visibility = 'Hidden'

label_text_modes_found = Label()

## Buttons
btn_calc = Button(description='CALCULATE MODES')
btn_calc.style.button_color = 'lightgray'

btn_plot = Button(description='PLOT EVERY MODE')
btn_plot.style.button_color = 'lightgray'
btn_plot.layout.visibility = 'Hidden'

btn_plot.label_gen_plots = Label(value = 'Generating plots...')
btn_plot.label_gen_plots.layout.visibility = 'Hidden'


def update_text(obj):
    V = 2 * pi / slider_lambda.value * slider_Na.value * slider_diam.value / 2
    if V > 30:
        label_V_max.layout.visibility = 'Visible'
        obj['owner'].value = obj['old']
        V = 2 * pi / slider_lambda.value * slider_Na.value * slider_diam.value / 2


    n_core = np.sqrt(slider_Na.value ** 2 + text_n_clad.value ** 2)
    label_n_core.value = f"{n_core:.4f}"
    label_V.value = f"{V:.4f}"
    btn_plot.layout.visibility = 'Hidden'



# Initialize
update_text(0)

## Observe Changes
slider_Na.observe(update_text, names = 'value')    
slider_lambda.observe(update_text, names = 'value')
slider_diam.observe(update_text, names = 'value')
text_n_clad.observe(update_text, names = 'value')
text_ng_clad.observe(update_text, names = 'value')

## Update Values
a = slider_diam.value / 2.0
Na = slider_Na.value
w = slider_lambda.value
V = 2 * pi / w * a * Na
n_clad = text_n_clad.value


# diam_core = 50
# Na=0.22 #typ. MM fiber

    


num_modes_show = 0

def btn_calc_eventhandler(obj):

    label_V_max.layout.visibility = 'Hidden'
    label_text_modes_found.value = 'Calculating...'

    w = slider_lambda.value
    diam_core = slider_diam.value
    Na = slider_Na.value
    n_cladding = text_n_clad.value
    ng_cladding = text_ng_clad.value
    n_core = np.sqrt(Na ** 2 + n_cladding ** 2)
    ng_core = n_core + (ng_cladding-n_cladding)
    delta_n = n_core-n_cladding
    a = diam_core / 2
    r = np.linspace(-3*a, 3*a, 100)
    

    (modes, tot_modes) = find_modes(a=a, Na=Na, w=w, n_cladding=n_cladding)
    
    delta_w = 1E-4
    delta_n_clad = (n_cladding-ng_cladding)/w*delta_w
    (modes_red, tot_modes_red) = find_modes(a=a, Na=Na, w=w+delta_w, n_cladding=n_cladding+delta_n_clad)
    (modes_blue, tot_modes_blue) = find_modes(a=a, Na=Na, w=w-delta_w, n_cladding=n_cladding-delta_n_clad)
    
    for ii,m in enumerate(modes):
        new_ng = m.neff-w*(modes_red[ii].neff-modes_blue[ii].neff)/(2*delta_w)
        m.ng = new_ng
    

    M_max = max(m.M for m in modes)
    M_list = np.arange(M_max)+1
    L_max = max(m.L for m in modes)
    modes.sort(key=lambda x: x.neff, reverse=True)
    
    label_text_modes_found.value = f'Distinct modes found: {len(modes)}. Total modes: {tot_modes}'
    obj.modes, obj.r = modes, r
    btn_plot.layout.visibility = 'visible'
    obj.mode_range_plot.clear_output(wait=True)
    with obj.mode_range_plot:
        fig, (ax1,ax2) = plt.subplots(1,2,figsize=(600 / dpi, 250 / dpi))
        for M in M_list:
            modes_subset= [mode_sub for mode_sub in modes if mode_sub.M==M]
            y=[m.neff for m in modes_subset]
            #x= L*np.ones(len(y))
            x=[m.L for m in modes_subset]
            ax1.plot(x,y,'.')
            y2=[m.ng for m in modes_subset]
            ax2.plot(x,y2,'.')
        
        
     
        
       
        ax1.set_ylim([n_cladding-delta_n*.15, n_core+delta_n*.15])
        ax1.axhline(n_cladding,linewidth=1, color='black', linestyle = 'dashed')
        ax1.axhline(n_core,linewidth=1, color='black',linestyle = 'dashed')
        ax1.annotate('n clad',xy = (0.03,0.04),xycoords='axes fraction', fontsize = 6)
        ax1.annotate('n core',xy = (0.03,0.91),xycoords='axes fraction', fontsize = 6)
        ax1.set_ylabel('$n_{eff}$',fontsize=15)
        ax1.set_xlabel('$L$')
        
        from matplotlib.ticker import MaxNLocator
        ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
        if L_max == 0:
            ax1.set_xticks([0.0])
        
        ax2.axhline(ng_core,linewidth=1, color='black',linestyle = 'dashed')
      
        trans = ax2.get_yaxis_transform()
   
        ylims = ax2.get_ylim()
        ax2.text(0.8,ng_core+0.05*(ylims[1]-ylims[0]),'ng core', transform = trans, fontsize = 6,verticalalignment = 'bottom')
        ax2.set_ylabel('$n_{g}$',fontsize=15)
        ax2.set_xlabel('$L$')
        ax2.xaxis.set_major_locator(MaxNLocator(integer=True))
        if L_max == 0:
            ax2.set_xticks([0.0])
        
        fig.tight_layout()
            
        plt.show(fig)
    
    


def btn_plot_eventhandler(obj):
    obj.label_gen_plots.layout.visibility = 'Visible'
    
    for ii in range(len(obj.mode_row_list)-1):
        obj.mode_row_list[ii + 1].children[1].clear_output(wait=True)
        obj.mode_fig_list[ii].clf()
        
    fig_titles_layout = Layout(width = '590px', border='None', margin = '10px 10px 10px 10px', justify_content = 'space-around')
    labels = [
        Label('  LP(L,M) mode description', layout=Layout(width='250px')),
        Label('Radial plot', layout=Layout(width='190px')),
        Label('2D image plot', layout=Layout(width='150px'))
    ]
    
    obj.mode_row_list = [HBox(labels, layout=fig_titles_layout)]
    
    obj.mode_fig_list = []
    
    modes, r = btn_calc.modes, btn_calc.r

    for ii in range(len(modes)):
        mode = modes[ii]
        
        show_text = f'<b>Mode {ii+1}</b>:<br>{mode.label}<br>Degeneracy {mode.degeneracy}' \
                    f'<br>neff = {mode.neff:0.6f}<br>ng = {mode.ng:0.6f}'
        
        text_label = HTML(value=show_text, layout=Layout(min_width='200px'))
        
        mode_row = HBox([text_label,Output()],layout=mode_row_layout)
        obj.mode_row_list.append(mode_row)
        
        with mode_row.children[1]:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(350 / dpi, 190 / dpi))
            
            ax1.plot(r, calc_Er(modes[ii], r))
            
            ax1.set_xticks([-mode.a, mode.a])
            ax1.set_xticklabels(['-a','a'])
            ax1.set_xlim([r[0],r[-1]])
            ax1.set_ylabel('E(r)')
            ax1.set_yticks([0])
            ax1.set_yticklabels([])
            ax1.set_position([0.1,0.1,.4,.8])
            
            E = calc_E_mesh(mode, r_mesh, phi_mesh, in_core_mesh, in_clad_mesh)
            
            ax2.imshow(E, cmap='RdBu', vmin = -1., vmax = 1.,
                       extent=(xx[0], xx[-1], yy[0], yy[-1]),
                       aspect='equal')
            
            ax2.plot(x_core_shape,y_core_shape,'black',linewidth=0.5)
            ax2.set_xticks([])
            ax2.set_yticks([])
            ax2.set_position([0.6, 0.1, .4, .8])
            
            plt.show(fig)
            obj.mode_fig_list.append(fig)
            
    obj.box_modes.children = obj.mode_row_list
    obj.label_gen_plots.layout.visibility = 'hidden'


gr_layout= Layout(width='800px', border='solid 1px',
                  grid_template_columns='20% 40% 20% 20%',
                  grid_template_rows='20% 20% 20% 20%',
                )

empty = Label()

label_layout = Layout(display='flex', justify_content='flex-end')

labels_units = [Label('Core diameter (um)', layout=label_layout), Label('Num. Aperture (NA)', layout=label_layout),
                Label('Wavelength (um)', layout=label_layout), btn_calc]

sliders = [slider_diam, slider_Na, slider_lambda, 
           label_text_modes_found]

labels_variables = [Label('n cladding', layout=label_layout),
                    Label('ng cladding', layout = label_layout),
                    Label('n core', layout=label_layout),
                    Label('V number', layout=label_layout)]

elements = [text_n_clad, text_ng_clad, label_n_core, HBox([label_V, label_V_max]), empty]

columns = [VBox(labels_units), VBox(sliders), VBox(labels_variables), VBox(elements)]

grid_ctrl = HBox(columns)

btn_calc.mode_range_plot = Output()

btn_plot_row = HBox([btn_plot, btn_plot.label_gen_plots])

mode_row_layout = Layout(width='590px', border='solid 1px', margin = '3px 3px 3px 3px', justify_content = 'space-around')

btn_plot.mode_row_list = []

mode_list_layout = Layout(width='620px', height='', flex_flow='column', display='flex')
btn_plot.box_modes = Box(children=btn_plot.mode_row_list, layout=mode_list_layout)

display(grid_ctrl)
display(btn_calc.mode_range_plot)
display(btn_plot_row)
display(btn_plot.box_modes)

btn_calc.modes = []
btn_calc.r=0.

btn_calc.on_click(btn_calc_eventhandler)
btn_plot.on_click(btn_plot_eventhandler)

Output()

Box(layout=Layout(display='flex', flex_flow='column', height='', width='620px'))

This is FIMOC version 2.1, published in December 2023.

Click here to go back to *[and there was light](https://andtherewaslight.github.io/fimoc/)*
